```txt
@ Purpose:
    image 파일에 대해서 cocomask 출력
@ Date
    24.02.14
@ Author
    유영재
@ Install Packages:
    pip install pycocotools numpy matplotlib scikit-image jupyter ipykernel 'opencv-python<4.6' tqdm
```

In [3]:
!python -m pip list | grep opencv

opencv-python             4.5.5.64


In [65]:
%matplotlib inline
from pycocotools.coco import COCO
import numpy as np
import skimage.io as io
import matplotlib.pyplot as plt
import os
from ipywidgets import interact
from tqdm import tqdm
import cv2

plt.rcParams['figure.figsize'] = [18, 12]

In [66]:
# [NEW_WANGSAN, NEW_KIMPO]
REGION = 'NEW_KIMPO'
ROOT_DIR = f'/data/NeuBoat/Batch1_3223_img/{REGION}'
IMAGE_FOLDER = os.path.join(ROOT_DIR, 'images')
LABEL_FILE = os.path.join(ROOT_DIR, 'labels', 'instances_default_output_file.json')
OUTPUT_FOLDER = os.path.join(ROOT_DIR, 'output')

In [67]:
if not os.path.exists(IMAGE_FOLDER):
    raise Exception('Image Folder not exists')

if not os.path.exists(LABEL_FILE):
    raise Exception('Label File not exists')

if not os.path.exists(OUTPUT_FOLDER):
    os.makedirs(OUTPUT_FOLDER)

https://github.com/cocodataset/cocoapi/blob/8c9bcc3cf640524c4c20a9c40e89cb6a2f2fa0e9/PythonAPI/pycocotools/coco.py#L20-L35

```txt
# The following API functions are defined:
#  COCO       - COCO api class that loads COCO annotation file and prepare data structures.
#  decodeMask - Decode binary mask M encoded via run-length encoding.
#  encodeMask - Encode binary mask M using run-length encoding.
#  getAnnIds  - Get ann ids that satisfy given filter conditions.
#  getCatIds  - Get cat ids that satisfy given filter conditions.
#  getImgIds  - Get img ids that satisfy given filter conditions.
#  loadAnns   - Load anns with the specified ids.
#  loadCats   - Load cats with the specified ids.
#  loadImgs   - Load imgs with the specified ids.
#  annToMask  - Convert segmentation in an annotation to binary mask.
#  showAnns   - Display the specified annotations.
#  loadRes    - Load algorithm results and create API for accessing them.
#  download   - Download COCO images from mscoco.org server.
# Throughout the API "ann"=annotation, "cat"=category, and "img"=image.
# Help on each functions can be accessed by: "help COCO>function".
```

In [68]:
cocoann = COCO(LABEL_FILE)

loading annotations into memory...
Done (t=0.07s)
creating index...
index created!


In [69]:
# 클래스(Category) id 가져오기
catIds = cocoann.getCatIds()
print('catIds:', catIds)
cats = cocoann.loadCats(catIds)
print('cats:', cats)

catIds: [1, 2, 3, 4, 5, 6]
cats: [{'id': 1, 'name': 'Jet Ski', 'supercategory': ''}, {'id': 2, 'name': 'Canoe', 'supercategory': ''}, {'id': 3, 'name': 'Buoy', 'supercategory': ''}, {'id': 4, 'name': 'Channel Marker', 'supercategory': ''}, {'id': 5, 'name': 'Pillar', 'supercategory': ''}, {'id': 6, 'name': 'Motor Boat', 'supercategory': ''}]


In [70]:
names = [cat['name'] for cat in cats]
print('names:', names)

names: ['Jet Ski', 'Canoe', 'Buoy', 'Channel Marker', 'Pillar', 'Motor Boat']


In [71]:
# image_id의 min, max  구하기
imgIds = cocoann.getImgIds()
print('imgIds:', imgIds)
print('min(imgIds):', min(imgIds))
print('max(imgIds):', max(imgIds))

imgIds: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220,

In [58]:
# interact 기반으로 image_id 선택해서 mask 그리기
@interact(image_id=(min(imgIds), max(imgIds)), mask=(0, 1))
def show_image(image_id, mask=True):
    img = cocoann.loadImgs(image_id)[0]
    I = io.imread(os.path.join(IMAGE_FOLDER, img['file_name']))
    plt.axis('off')
    plt.imshow(I)
    
    if mask:
        annIds = cocoann.getAnnIds(image_id, catIds=catIds, iscrowd=None)
        anns = cocoann.loadAnns(annIds)
        cocoann.showAnns(anns)
    
    plt.show()

interactive(children=(IntSlider(value=156, description='image_id', max=312, min=1), IntSlider(value=1, descrip…

### Save Image-Label Masks

In [72]:
### Save Images

for image_id in tqdm(imgIds):
    img = cocoann.loadImgs(image_id)[0]
    I = io.imread(os.path.join(IMAGE_FOLDER, img['file_name']))
    print('I shape:', I.shape)
    
    # height가 width보다 큰 경우, 이미지를 반시계 방향으로 90도 회전
    if I.shape[0] > I.shape[1]:
        I = cv2.rotate(I, cv2.ROTATE_90_COUNTERCLOCKWISE)
        print('Rotated shape:', I.shape)
        
    plt.figure(figsize=(10, 10))
    plt.axis('off')
    plt.imshow(I)
    
    annIds = cocoann.getAnnIds(image_id, catIds=catIds, iscrowd=None)
    anns = cocoann.loadAnns(annIds)
    cocoann.showAnns(anns)
    plt.savefig(os.path.join(OUTPUT_FOLDER, img['file_name']))
    # print('Save:', os.path.join(OUTPUT_FOLDER, img['file_name']))
    plt.close()

  0%|          | 1/312 [00:00<00:55,  5.62it/s]

I shape: (1080, 1920, 3)
I shape: (1080, 1920, 3)


  1%|          | 3/312 [00:00<00:48,  6.39it/s]

I shape: (1080, 1920, 3)
I shape: (1080, 1920, 3)


  2%|▏         | 5/312 [00:00<00:46,  6.60it/s]

...

 99%|█████████▉| 309/312 [01:11<00:00,  6.54it/s]

I shape: (1080, 1920, 3)
I shape: (1080, 1920, 3)


100%|█████████▉| 311/312 [01:11<00:00,  6.50it/s]

I shape: (1080, 1920, 3)
I shape: (1080, 1920, 3)


100%|██████████| 312/312 [01:11<00:00,  4.35it/s]
